In [6]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
#from torchviz import make_dot Need tensorflow and some other stuff to be installed 
#bc torch doesn't have visualisation tool itself
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
#Get X and Y data created:
np.random.seed(42)
x = np.random.rand(100, 1)
true_a, true_b = 1, 2
y = true_a + true_b*x + 0.1*np.random.randn(100, 1)

x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float()

#Create Custom Dataset that inherits torch's dataset
class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)
    
#For simple datdasets use TensorDataset directly, avoid custom datasets
dataset = TensorDataset(x_tensor, y_tensor) # dataset = CustomDataset(x_tensor, y_tensor)

train_dataset, val_dataset = random_split(dataset, [80, 20])

train_loader = DataLoader(dataset=train_dataset, batch_size=16)
val_loader = DataLoader(dataset=val_dataset, batch_size=20)

In [10]:
class ManualLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(y, yhat)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

# Estimate a and b
torch.manual_seed(42)

model = ManualLinearRegression().to(device) # model = nn.Sequential(nn.Linear(1, 1)).to(device)
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=1e-1)
train_step = make_train_step(model, loss_fn, optimizer)

n_epochs = 40
training_losses = []
validation_losses = []

print(model.state_dict())

for epoch in range(n_epochs): #epoch prolazi jednom kroz CEO dataset
    
    #Za sve batch-eve do:
    batch_losses = []
    for x_batch, y_batch in train_loader:
        
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        loss = train_step(x_batch, y_batch)
        
        batch_losses.append(loss)
        
    training_loss = np.mean(batch_losses)
    training_losses.append(training_loss)

    with torch.no_grad():
        val_losses = []
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            model.eval()
            yhat = model(x_val)
            val_loss = loss_fn(y_val, yhat).item()
            val_losses.append(val_loss)
        validation_loss = np.mean(val_losses)
        validation_losses.append(validation_loss)

    print(f"[{epoch+1}] Training loss: {training_loss:.3f}\t Validation loss: {validation_loss:.3f}")

print(model.state_dict())

OrderedDict([('linear.weight', tensor([[0.7645]])), ('linear.bias', tensor([0.8300]))])
[1] Training loss: 0.326	 Validation loss: 0.122
[2] Training loss: 0.074	 Validation loss: 0.072
[3] Training loss: 0.056	 Validation loss: 0.060
[4] Training loss: 0.050	 Validation loss: 0.053
[5] Training loss: 0.045	 Validation loss: 0.047
[6] Training loss: 0.040	 Validation loss: 0.042
[7] Training loss: 0.036	 Validation loss: 0.038
[8] Training loss: 0.032	 Validation loss: 0.034
[9] Training loss: 0.029	 Validation loss: 0.031
[10] Training loss: 0.026	 Validation loss: 0.028
[11] Training loss: 0.024	 Validation loss: 0.026
[12] Training loss: 0.022	 Validation loss: 0.024
[13] Training loss: 0.020	 Validation loss: 0.022
[14] Training loss: 0.018	 Validation loss: 0.020
[15] Training loss: 0.017	 Validation loss: 0.019
[16] Training loss: 0.016	 Validation loss: 0.018
[17] Training loss: 0.015	 Validation loss: 0.016
[18] Training loss: 0.014	 Validation loss: 0.016
[19] Training loss: 0